In [2]:
import torch
import torch.nn as nn
from torch.nn import BatchNorm2d
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda


In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

batch_size = 64
#train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
#test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

x_test, y_test = torch.utils.data.random_split(train_dataset, [50000, 10000])

xtrainloader = torch.utils.data.DataLoader(dataset=x_test, batch_size=batch_size, shuffle=True)

for _, (i,l) in enumerate(xtrainloader):
    #i.reshape(-1,28,28,1)  
    print(i.shape, l.shape)


torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size

In [4]:
class SqueezeExciteBlock2D(nn.Module):
    def __init__(self, filters):
        super(SqueezeExciteBlock2D, self).__init__()
        self.filters = filters
        self.weight1 = nn.Parameter(torch.empty(self.filters, self.filters // 32))
        self.bias1 = nn.Parameter(torch.empty(1, self.filters // 32))
        self.weight2 = nn.Parameter(torch.empty(self.filters // 32, self.filters))
        self.bias2 = nn.Parameter(torch.empty(1, self.filters))
        self.avgPool2d = nn.AvgPool2d(1)
        
        # Initialize weights and biases
        nn.init.xavier_uniform_(self.weight1)
        nn.init.zeros_(self.bias1)
        nn.init.xavier_uniform_(self.weight2)
        nn.init.zeros_(self.bias2)

    def forward(self, x):
        print(x.shape)
        se = F.avg_pool2d(x, kernel_size=(14,14))
        print(se.shape)
        se = se.reshape(-1, self.filters)
        print(se.shape)
        se = torch.mm(se, self.weight1) + self.bias1
        print(se.shape)
        se = F.relu(se)
        se = torch.mm(se, self.weight2) + self.bias2
        print(se.shape)
        se = torch.sigmoid(se).reshape(-1, self.filters, 1, 1)
        print(se.shape)

        se = x * se
        return se




In [5]:
class ConvBlock(nn.Module):
    def __init__(self, h, w, inp, filters = 128, conv = 3):
        super(ConvBlock, self).__init__()
        self.h, self.w, self.inp = h, w, inp
        self.cweights = nn.ParameterList([
            nn.Parameter(torch.empty(filters, inp if i == 0 else filters, conv , conv))
            for i in range(3)
        ])
        self.cbiases = nn.ParameterList([
            nn.Parameter(torch.empty(filters))
            for i in range(3)
        ])

        for weight in self.cweights:
            nn.init.xavier_uniform_(weight)
        for bias in self.cbiases:
            nn.init.zeros_(bias)

        self._bn = nn.BatchNorm2d(128)
        self._seb = SqueezeExciteBlock2D(filters)

    def forward(self, input):
        x = input.reshape((-1, self.inp, self.w, self.h))
        for cweight, cbias in zip(self.cweights, self.cbiases):
            x = F.pad(x, (1,1,1,1))
            x = F.conv2d(x, cweight, bias=cbias)
            x = F.relu(x)
        x = self._bn(x)
        x = self._seb(x)
        return x
    


In [6]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv = nn.ModuleList([ConvBlock(28,28,1), ConvBlock(28,28,128), ConvBlock(14,14,128)])
        self.weight1 = nn.Parameter(torch.empty(128,10))
        self.weight2 = nn.Parameter(torch.empty(128,10))
        nn.init.xavier_uniform_(self.weight1)
        nn.init.xavier_uniform_(self.weight2)
    
    def forward(self, x):
        print(x.shape)
        x = self.conv[0](x)
        print(x.shape)
        x = self.conv[1](x)
        print(x.shape)
        x = F.avg_pool2d(x, kernel_size=(2,2))
        print(x.shape)
        x = self.conv[2](x)
        x1 = F.avg_pool2d(x, kernel_size=(14, 14)).reshape((-1, 128))
        x2 = F.max_pool2d(x, kernel_size=(14, 14)).reshape((-1, 128))
        xo = torch.mm(x1, self.weight1) + torch.mm(x2, self.weight2)
        print(xo.shape)
        return xo
    

In [7]:
import sys

if __name__ == "__main__":

    QUICK = False

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

    train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

    batch_size = 64
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    lrs = [1e-4, 1e-5] if QUICK else [1e-3, 1e-4, 1e-5, 1e-5]
    epochss = [2, 1] if QUICK else [13, 3, 3, 1]
    BS = 32
    lmbd = 0.00025

    def lossfn(out, y):
        criterion = nn.CrossEntropyLoss()
        regularization_term = lmbd * (model.weight1.abs().sum() + model.weight2.abs().sum())
        return criterion(out, y) + regularization_term

    # Create the BigConvNet model
    model = ConvNet()

    if len(sys.argv) > 1:
        try:
            checkpoint = torch.load(sys.argv[1])
            model.load_state_dict(checkpoint)
            print('Loaded weights "' + sys.argv[1] + '", evaluating...')
            # Add evaluation code here
        except:
            print('Could not load weights "' + sys.argv[1] + '".')


    # Define the optimizer
    for lr, epochs in zip(lrs, epochss):
        optimizer = optim.Adam(model.parameters(), lr=lr)
        for epoch in range(1, epochs + 1):
            for batch_idx, (data, target) in enumerate(train_loader):
                #data, target = data.to(device), target.to(device)  # Move data to GPU if needed'
                optimizer.zero_grad()
                output = model(data)
                loss = lossfn(output, target)
                loss.backward()
                optimizer.step()
            accuracy = model.eval()  # Evaluate using test_loader
            model.save(f'examples/checkpoint{accuracy * 1e6:.0f}')

    

Could not load weights "--f=/home/jkr/.local/share/jupyter/runtime/kernel-v2-119411nahBvH7e2fHl.json".
torch.Size([64, 1, 28, 28])
torch.Size([64, 128, 28, 28])
torch.Size([64, 128, 2, 2])
torch.Size([256, 128])
torch.Size([256, 4])
torch.Size([256, 128])
torch.Size([256, 128, 1, 1])


RuntimeError: The size of tensor a (64) must match the size of tensor b (256) at non-singleton dimension 0